In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from buffer import ReplayBuffer
from PPO import PPO
import gym
from Discriminator import Discriminator
from reward_env import RewardEnv
import network_sim
import contextlib

In [2]:
device='cuda'
BUFFER_SIZE=8192
hidden_dim=32

In [3]:
class CustomEnv:
    def __init__(self,env_id):
        self.env=gym.make(env_id)
        self.student_buffer=ReplayBuffer(8192)
        self.teacher_buffer=ReplayBuffer(8192)
        
    def reset(self):
        return self.env.reset()
    
    def step(self,action):
        with contextlib.redirect_stdout(None):
            return self.env.step(action)
    
    def get_state_dim(self):
        return self.env.observation_space.shape
    
    def get_action_dim(self):
        return self.env.action_space.shape

In [4]:
class StudentAgent:
    def __init__(self,state_dim,action_dim,env:CustomEnv):
        self.env=env
        self.replay_buffer=self.env.student_buffer
        self.model=PPO(state_dim,action_dim,self.replay_buffer)
        
    def generate_trajectory(self,step:int):
        pb=tqdm(range(step))
        num=0
        for i in pb:
            s=self.env.reset()
            d=False
            while not d:
                a,_,l,v=self.model.select_action(s)
                s_,r,d,_=self.env.step(a)
                # print(type(r))
                self.model.buffer.store(s,a,l,s_,r,v,d)
                num+=1
            #pb.update()
        print('生成',num,'条轨迹')   
        
    def train(self,total_timestep,batch_size):
        pb=tqdm(range(total_timestep))
        for i in pb:
            self.model.update(batch_size)

In [5]:
env=CustomEnv('PccNs-v0')

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


/home/data/envs/IL/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
sa=StudentAgent(np.prod(env.get_state_dim()),env.get_action_dim()[0],env)

In [7]:
sa.generate_trajectory(100)

  0%|          | 0/100 [00:00<?, ?it/s]

Reward: 0.00, Ewma Reward: 0.00


  1%|          | 1/100 [00:03<05:33,  3.37s/it]

Reward: -1651.17, Ewma Reward: -16.51


  2%|▏         | 2/100 [00:07<05:51,  3.59s/it]

...

Reward: 1278.86, Ewma Reward: -9.61


 99%|█████████▉| 99/100 [02:23<00:00,  1.24it/s]

Reward: 1327.15, Ewma Reward: 3.76


100%|██████████| 100/100 [02:26<00:00,  1.46s/it]

生成 40000 条轨迹


In [8]:
sa.train(10,2048)

100%|██████████| 10/10 [00:03<00:00,  2.70it/s]


In [9]:
class TeacherAgent():
    def __init__(self,state_dim,action_dim,env:CustomEnv):
        self.trajectory_buffer=env.student_buffer
        self.replay_buffer=env.teacher_buffer
        self.model=PPO(state_dim+action_dim,1,self.replay_buffer)
        self.discriminator=Discriminator(state_dim+action_dim,hidden_dim,64,self.trajectory_buffer,self.trajectory_buffer)
        
    def ComputeReward(self):
        pb=tqdm(range(min(self.trajectory_buffer.index,self.trajectory_buffer.buffer_size)))
        for i in pb:
            sa_pair=torch.cat((self.trajectory_buffer.state[i],self.trajectory_buffer.action[i]),-1)
            reward,_,l,v=self.model.select_action(sa_pair)
            self.replay_buffer.store(sa_pair,
                                     reward,
                                     l,
                                     self.trajectory_buffer.next_state[i],
                                     self.discriminator.model(sa_pair).detach().cpu().numpy(),
                                     v,
                                     self.trajectory_buffer.done[i],
                                     )
            self.trajectory_buffer.reward[i]=reward
            pb.update()
        self.discriminator.collect_expert()
            
    def trainPPO(self,total_timestep:int):
        for i in range(total_timestep):
            self.model.update(1024)
            
    def trainDiscriminator(self,total_timestep:int):
        self.discriminator.update(total_timestep,False)
        
    def train(self,total_timestep:int,PPO_timestep:int,D_timestep:int):
        pb=tqdm(range(total_timestep))
        for i in pb:
            self.trainDiscriminator(D_timestep)
            pb.update()
            self.trainPPO(PPO_timestep)
            pb.update()
        

In [10]:
ta=TeacherAgent(np.prod(env.get_state_dim()),env.get_action_dim()[0],env)

In [11]:
ta.ComputeReward()

100%|██████████| 8192/8192 [00:05<00:00, 1581.68it/s]


In [12]:
ta.trainDiscriminator(100)

In [13]:
ta.trainPPO(100)

In [14]:
for i in range(10):
    sa.train(10,1024)
    ta.train(10,5,5)

  0%|          | 0/10 [00:00<?, ?it/s]/home/tools/DR/MyProject/RILE/PPO.py:184: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
/home/data/envs/IL/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1024, 1])) that is different to the input size (torch.Size([1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


In [15]:
rewards=[]
for i in range(100):
    s=env.reset()
    d=False
    reward=0
    while not d:
        a,_,_,_=sa.model.select_action(s)
        s_,r,d,_=env.step(a)
        reward+=r
    rewards.append(reward)

Reward: -1616.16, Ewma Reward: -12.44
Reward: -1475.92, Ewma Reward: -27.08
Reward: 523.15, Ewma Reward: -21.57
Reward: 57.40, Ewma Reward: -20.78
Reward: -104.62, Ewma Reward: -21.62
Reward: -317.19, Ewma Reward: -24.58
Reward: 987.47, Ewma Reward: -14.46
Reward: -722.73, Ewma Reward: -21.54
Reward: -1385.88, Ewma Reward: -35.18
Reward: 535.77, Ewma Reward: -29.48
Reward: 562.63, Ewma Reward: -23.55
Reward: -618.77, Ewma Reward: -29.51
Reward: -863.98, Ewma Reward: -37.85
Reward: 236.24, Ewma Reward: -35.11
Reward: 1087.96, Ewma Reward: -23.88
Reward: 195.06, Ewma Reward: -21.69
Reward: -261.17, Ewma Reward: -24.08
Reward: 174.48, Ewma Reward: -22.10
Reward: -418.33, Ewma Reward: -26.06
Reward: -267.37, Ewma Reward: -28.47
Reward: -333.26, Ewma Reward: -31.52
Reward: 601.25, Ewma Reward: -25.19
Reward: -640.03, Ewma Reward: -31.34
Reward: 408.71, Ewma Reward: -26.94
Reward: -976.63, Ewma Reward: -36.44
Reward: 917.98, Ewma Reward: -26.90
Reward: -233.88, Ewma Reward: -28.96
Reward: -9

In [16]:
np.mean(reward)

-306.07449684345556

In [20]:
env.env.reset()

Reward: 0.00, Ewma Reward: -50.22


array([0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.])